**Check environment dependencies**

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"
! python3 -c "import sklearn; print('Sklearn version: {}'.format(sklearn.__version__))"

In [ ]:
from datetime import datetime

import google.cloud.aiplatform as aip
from google.cloud import aiplatform
import kfp
from kfp.v2 import dsl, compiler
from kfp.v2.google.client import AIPlatformClient

# custom code for data processing and model training
from utils import create_data, train_model

**Define environment variables**

User should update the <code>BUCKET_NAME</code>. The <code>PROJECT_ID</code> is picked up based on the gcloud configuration

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "black-friday-dataset-test"  # modify
BUCKET_URI = f"gs://{BUCKET_NAME}"
REGION = "us-central1"
PIPELINE_ROOT = "{}/pipeline_root/black_friday".format(BUCKET_URI)
DISPLAY_NAME = "black-friday-" + TIMESTAMP
PACKAGE_PATH = "pipeline.json"
project_id_shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = project_id_shell_output[0]

## Data processing and model training

**Initialize the client**

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

**Define the pipeline**

In [ ]:
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT,
    name="metadata-pipeline-v2",
)
def pipeline(
    train_file_x: str,
    train_file_y: str,
    test_file_x: str,
    test_file_y: str,
    best_params_file: str,
    metrics_file: str,
    num_iterations: int,
    hp_tune: bool,
):

    create_data_task = create_data(
        project_id="mwpmltr", bucket_name=BUCKET_NAME, dataset_id="black_friday"
    )

    train_model_task = train_model(
        hp_tune=hp_tune,
        project_id="mwpmltr",
        bucket_name=BUCKET_NAME,
        num_iterations=num_iterations,
        train_file_x=create_data_task.outputs["train_file_x"],
        test_file_x=create_data_task.outputs["test_file_x"],
        train_file_y=create_data_task.outputs["train_file_y"],
        test_file_y=create_data_task.outputs["test_file_y"],
    )

In [ ]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path=PACKAGE_PATH)

**Submit the pipeline to Vertex AI Pipeline**

In [ ]:
job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path=PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "train_file_x": "x_train.csv",
        "train_file_y": "y_train.csv",
        "test_file_x": "x_test.csv",
        "test_file_y": "y_test.csv",
        "best_params_file": "best_params.json",
        "metrics_file": "metrics.json",
        "num_iterations": 2,
        "hp_tune": True,
    },
)

job.run()

**Copy the model to your local directory**

Get the URI of the model file by clicking through the Vertex AI Pipeline UI. Select the model artifact and follow the path to the model

In [ ]:
! gsutil cp gs://black-friday-dataset-test/pipeline_root/black_friday/55590906972/metadata-pipeline-v2-20221103004839/train-model_-102004442487848960/model_file.pkl model.pkl

## Model Deployment

**Set environment variables for command line arguments**

For custom prediction routines, a docker image must be provided

In [ ]:
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env REPOSITORY=black-friday-v1
%env IMAGE=black-friday-image

In [ ]:
# build image
!docker build --tag=$REGION-docker.pkg.dev/$PROJECT_ID/$REPOSITORY/$IMAGE .

In [ ]:
# create repository in artifact repository
! gcloud artifacts repositories create $REPOSITORY  \
                             --repository-format=docker \
                             --location=$REGION

In [ ]:
# push docker image to the newly created artifact repository
! docker push $REGION-docker.pkg.dev/$PROJECT_ID/$REPOSITORY/$IMAGE

In [ ]:
# upload model to Vertex AI  model registry
! gcloud ai models upload \
  --region=us-central1 \
  --display-name=black-friday-model \
  --container-image-uri=$REGION-docker.pkg.dev/$PROJECT_ID/$REPOSITORY/$IMAGE \
  --container-ports=5005 \
  --container-health-route=/healthz \
  --container-predict-route=/predict

In [ ]:
# list models to double check
!gcloud ai models list \
  --region=us-central1 \
  --filter=display_name=black-friday-model

In [ ]:
# create a Vertex AI endpoint
!gcloud ai endpoints create \
  --region=us-central1 \
  --display-name=black-friday-test2

**Deploy the model to the endpoint**

The model endpoint is collected from the previous cell. The model id is collected from the list of models

In [ ]:

!gcloud ai endpoints deploy-model 4126638662857261056 \
  --region=us-central1 \
  --model=7845406890321248256 \
  --display-name=black-friday-model \
  --machine-type=n1-standard-4 \
  --min-replica-count=1 \
  --max-replica-count=2 